## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.sample(10)

,date,symbol,open,close,low,high,volume
612709,2015-02-06,T,34.490002,34.869999,34.410000,35.049999,27709200.0
289820,2012-06-13,PSA,135.490005,135.690002,135.089996,137.479996,562700.0
519989,2014-05-08,STI,38.110001,37.939999,37.799999,38.270000,3119700.0
578852,2014-10-28,WM,48.330002,48.419998,48.040001,48.419998,1391100.0
284989,2012-05-30,HUM,77.110001,76.820000,76.639999,77.830002,1282900.0
670660,2015-07-28,ALB,49.900002,50.779999,49.590000,50.970001,893700.0
202962,2011-09-19,VZ,36.410000,36.279999,36.029999,36.410000,12186800.0
235986,2011-12-29,COF,41.980000,42.639999,41.820000,42.680000,4079000.0
817402,2016-09-26,DPS,91.500000,90.290001,90.199997,91.510002,1178300.0
21157,2010-03-09,NUE,44.509998,44.930000,44.310001,45.320000,4952400.0


### Drop columns `date` and  `symbol`

In [7]:
data.drop(columns=['date','symbol'],inplace=True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [9]:
data_m=data.iloc[0:1000,:]
data_m['volume']=data_m['volume']/1000000
data_m.head()

C:\Users\sinamy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [10]:
y=data_m['volume']
x=data_m.iloc[:,0:4]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x.values, y.values, test_size=0.2,random_state=456)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(800, 4) (800,)
(200, 4) (200,)


#### Convert Training and Test Data to numpy float32 arrays


In [11]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [35]:
from sklearn.preprocessing import Normalizer
X_train = Normalizer().fit_transform(X_train)
X_test = Normalizer().transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [45]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [37]:
class Model():
      def __init__(self):
        self.W = W
        self.B = b 
        self.variables = [ self.W, self.B ]
    
      def frwrd_pass(self,X_train):
        out = tf.matmul( X_train, self.W ) + self.B
        return out

3.Loss (Cost) Function [Mean square error]

In [38]:
#loss = tf.reduce_mean(tf.square(y-y_train),name='Loss')
def loss(predicted_y, desired_y):
      return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [46]:
def train(X_train, Y_train, epochs,test):
    print(test)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.05)
    for i in range(epochs):
        with tf.GradientTape() as tape:
            predicted = model.frwrd_pass(X_train)
            curr_loss = loss(predicted, Y_train)
            grads = tape.gradient( curr_loss, model.variables )
            optimizer.apply_gradients(zip(grads, model.variables),global_step=tf.train.get_or_create_global_step())
            if test == 0 :
                print("Loss at step {:d}: {:.3f}".format(i, loss(model.frwrd_pass( X_train ), Y_train)))
            if test == 1 :
                if i % 5 == 0:
                     print("Loss at step {:d}: {:.3f}".format(i, loss(model.frwrd_pass( X_train ), Y_train)))

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [47]:
#how many times data need to be shown to model
model = Model()
training_epochs = 100
train(X_train,y_train,training_epochs,0)

0
Loss at step 0: 160.815
Loss at step 1: 154.790
Loss at step 2: 150.933
Loss at step 3: 148.465
Loss at step 4: 146.886
Loss at step 5: 145.875
Loss at step 6: 145.228
Loss at step 7: 144.814
Loss at step 8: 144.549
Loss at step 9: 144.379
Loss at step 10: 144.271
Loss at step 11: 144.201
Loss at step 12: 144.157
Loss at step 13: 144.128
Loss at step 14: 144.110
Loss at step 15: 144.098
Loss at step 16: 144.091
Loss at step 17: 144.086
Loss at step 18: 144.083
Loss at step 19: 144.081
Loss at step 20: 144.080
Loss at step 21: 144.079
Loss at step 22: 144.078
Loss at step 23: 144.078
Loss at step 24: 144.078
Loss at step 25: 144.078
Loss at step 26: 144.078
Loss at step 27: 144.078
Loss at step 28: 144.078
Loss at step 29: 144.078
Loss at step 30: 144.078
Loss at step 31: 144.078
Loss at step 32: 144.077
Loss at step 33: 144.078
Loss at step 34: 144.078
Loss at step 35: 144.078
Loss at step 36: 144.078
Loss at step 37: 144.078
Loss at step 38: 144.078
Loss at step 39: 144.078
Loss at 

In [126]:
train(X_test,y_test,training_epochs,1)

1
Loss at step 0: 468.116
Loss at step 5: 468.116
Loss at step 10: 468.116
Loss at step 15: 468.116
Loss at step 20: 468.116
Loss at step 25: 468.116
Loss at step 30: 468.116
Loss at step 35: 468.116
Loss at step 40: 468.116
Loss at step 45: 468.116
Loss at step 50: 468.116
Loss at step 55: 468.116
Loss at step 60: 468.116
Loss at step 65: 468.116
Loss at step 70: 468.116
Loss at step 75: 468.116
Loss at step 80: 468.116
Loss at step 85: 468.116
Loss at step 90: 468.116
Loss at step 95: 468.116


### Get the shapes and values of W and b

In [48]:
print('W:',W)
print('b:',b)

W: <tf.Variable 'Weights:0' shape=(4, 1) dtype=float32, numpy=
array([[1.2768772],
       [1.2809805],
       [1.2637109],
       [1.2921484]], dtype=float32)>
b: <tf.Variable 'Bias:0' shape=(1,) dtype=float32, numpy=array([2.557002], dtype=float32)>


In [49]:
y_test_pred = tf.add(tf.matmul(X_test,W),b,name='output')

In [50]:
loss(y_test_pred, y_test)

<tf.Tensor: id=36753, shape=(), dtype=float32, numpy=469.1192>

### Model Prediction on 1st Examples in Test Dataset

In [152]:
y_test_pred[0,:]

<tf.Tensor: id=141502, shape=(1,), dtype=float32, numpy=array([5.0276537], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [3]:
import pandas as pd
df = pd.read_csv('Iris.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [6]:
df=pd.get_dummies(df)
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [7]:
x = df.iloc[:,1:5]
y = df.iloc[:,5:8]

### Divide the dataset into Training and test (70:30)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3,random_state=456)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(105, 4) (105, 3)
(45, 4) (45, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [27]:
# Create simple network with Keras
from keras.models import Sequential
from keras.layers import Dense
tf.disable_eager_execution()
# Extending the first model with activation functions
model = Sequential()
#specifying activation functions
model.add(Dense(3, input_dim=4, init='uniform', activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

C:\Users\sinamy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, input_dim=4, activation="softmax", kernel_initializer="uniform")`
  


### Model Training 

In [28]:
# Fit the model
model.fit(X_train, y_train, nb_epoch=20, batch_size=10,  verbose=2)
# evaluate the model
scores = model.evaluate(X_train, y_train)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

C:\Users\sinamy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
 - 1s - loss: 1.0350 - acc: 0.4571
Epoch 2/20
 - 0s - loss: 0.9527 - acc: 0.6952
Epoch 3/20
 - 0s - loss: 0.8946 - acc: 0.7333
Epoch 4/20
 - 0s - loss: 0.8375 - acc: 0.7143
Epoch 5/20
 - 0s - loss: 0.8160 - acc: 0.6667
Epoch 6/20
 - 0s - loss: 0.7622 - acc: 0.7333
Epoch 7/20
 - 0s - loss: 0.7366 - acc: 0.7143
Epoch 8/20
 - 0s - loss: 0.7054 - acc: 0.7333
Epoch 9/20
 - 0s - loss: 0.6913 - acc: 0.7143
Epoch 10/20
 - 0s - loss: 0.6601 - acc: 0.7143
Epoch 11/20
 - 0s - loss: 0.6451 - acc: 0.7143
Epoch 12/20
 - 0s - loss: 0.6267 - acc: 0.7143
Epoch 13/20
 - 0s - loss: 0.6184 - acc: 0.7333
Epoch 14/20
 - 0s - loss: 0.6077 - acc: 0.7429
Epoch 15/20
 - 0s - loss: 0.5944 - acc: 0.7143
Epoch 16/20
 - 0s - loss: 0.5775 - acc: 0.7905
Epoch 17/20
 - 0s - loss: 0.5698 - acc: 0.7429
Epoch 18/20
 - 0s - loss: 0.5571 - acc: 0.7143
Epoch 19/20
 - 0s - loss: 0.5582 - acc: 0.7524
Epoch 20/20
 - 0s - loss: 0.5253 - acc: 0.9143
105/105 [==============================] - 0s 1ms/step
acc: 71.43%


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


### Model Prediction

In [30]:
scores = model.evaluate(X_test, y_test)
print(scores)
pred = model.predict(X_test)
pred

45/45 [==============================] - 0s 119us/step
[0.747825645075904, 0.5555555635028415]


array([[0.03944512, 0.20212527, 0.75842965],
       [0.12312265, 0.25942904, 0.6174484 ],
       [0.7618241 , 0.13750218, 0.10067376],
       [0.15808238, 0.25812274, 0.5837949 ],
       [0.7904687 , 0.12202542, 0.08750585],
       [0.04872273, 0.21643433, 0.7348429 ],
       [0.02751605, 0.17681377, 0.7956702 ],
       [0.22466563, 0.2660287 , 0.50930566],
       [0.05084636, 0.20588836, 0.7432653 ],
       [0.13958418, 0.25961164, 0.60080415],
       [0.09598118, 0.24448916, 0.6595297 ],
       [0.14990285, 0.2596133 , 0.59048384],
       [0.742519  , 0.1444818 , 0.1129992 ],
       [0.09818966, 0.25220105, 0.64960927],
       [0.05491167, 0.2080596 , 0.7370287 ],
       [0.04340836, 0.2036572 , 0.7529345 ],
       [0.03737795, 0.17436884, 0.7882532 ],
       [0.15099038, 0.2717921 , 0.57721746],
       [0.82453316, 0.10189032, 0.07357655],
       [0.11126143, 0.24742736, 0.6413112 ],
       [0.02233278, 0.15387413, 0.8237931 ],
       [0.12035813, 0.24832767, 0.63131416],
       [0.

### Save the Model

In [31]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?